### Step1
##### #Follow the website [link](https://www.learncodewithmike.com/2020/02/python-email.html).
##### #Search "應用程式密碼" in 步驟4 of the website.
##### #Get your own password

In [ ]:
# %pip install tqdm
# %pip install pandas
# %pip install openpyxl

In [53]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import pandas as pd
from tqdm import tqdm
from openpyxl import load_workbook

### Step2 (here)
##### #Fill in your own information.
##### #Change the path of the excel.
##### #Modify the content if needed.

In [ ]:
#原始數據
#學號	姓名	1	2	3	4	5	6	7	8	9	10	總分
excel_path=r"D:\Download\grade_temp.xlsx"#here

data = pd.read_excel(excel_path)
data

In [ ]:
num=len(data)
print(num)
# 取得第一筆資料
first_row = data.iloc[0]
# 將第一筆資料轉為字典格式
first_row_data = first_row.to_dict()
print(first_row_data)

In [48]:
#讀取附註
wb = load_workbook(excel_path)

# 選擇工作表
sheet = wb.active

comment_dict={}
for i in range(2,num+2):
    student_id=str(sheet[f'A{i}'].value)
    for row in sheet[f'C{i}':f'L{i}']:
        comment_list=[]
        for cell in row:
            # 檢查是否有附註
            if cell.comment:
                comment_list.append(cell.comment.text.split(':')[1][1:])
            else:
                comment_list.append("")
        comment_dict[student_id]=comment_list

In [49]:
comment_dict['411123050']

['', '', '', '算式間多空一格', '上傳成第四題', '', '', '', 'output最後一行無空行', '']

In [ ]:
# 配置發件人
sender_email = "here@gms.ndhu.edu.tw"  #your gmail
sender_password = "here"  # password without space

In [ ]:
for i in  tqdm(range(num), desc="Processing"):
    info=data.iloc[i]
    info=info.to_dict()
    id=str(info["學號"])
    recipient_email=id+"@gms.ndhu.edu.tw"
    name=str(info["姓名"])
    numbers=[]
    numbers = [str(info[index]) for index in range(1, 11)]
    grade=str(info["總分"])
    
    # 創建郵件對象
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = "here"

    # HTML 內容
    html_content = """
    <html>
    <head>
        <style>
            h1 {
                font-size: 16px;  /* 增大標題文字大小 */
                font-weight: bold;
            }

            table {
                border-collapse: collapse;
                width: 100%;
            }
            th, td {
                border: 1px solid black;
                padding: 10px;
                text-align: center;
            }
            th {
                background-color: #f2f2f2;
            }

            h3 {
                font-size: 20px;  /* 增大標題文字大小 */
                font-weight: bold;
            }

            ul {
                font-size: 16px;  /* 增大列表文字大小 */
            }        
        </style>
    </head>
    <body>
        <h2>成績單</h2>
        <table>
            <tr>
                <th>學號</th>
                <th>姓名</th>
                <th>1</th>
                <th>2</th>
                <th>3</th>
                <th>4</th>
                <th>5</th>
                <th>6</th>
                <th>7</th>
                <th>8</th>
                <th>9</th>
                <th>10</th>
                <th>總分</th>
            </tr>
    """+f"""
            <tr>
                <td>{id}</td>
                <td>{name}</td>
                <td>{numbers[0]}</td>
                <td>{numbers[1]}</td>
                <td>{numbers[2]}</td>
                <td>{numbers[3]}</td>
                <td>{numbers[4]}</td>
                <td>{numbers[5]}</td>
                <td>{numbers[6]}</td>
                <td>{numbers[7]}</td>
                <td>{numbers[8]}</td>
                <td>{numbers[9]}</td>
                <td>{grade}</td>
            </tr>
    """+f"""</table>
        <h3>評語</h3>
        <ul>
            <li>第1題: {comment_dict[id][0]}</li>
            <li>第2題: {comment_dict[id][1]}</li>
            <li>第3題: {comment_dict[id][2]}</li>
            <li>第4題: {comment_dict[id][3]}</li>
            <li>第5題: {comment_dict[id][4]}</li>
            <li>第6題: {comment_dict[id][5]}</li>
            <li>第7題: {comment_dict[id][6]}</li>
            <li>第8題: {comment_dict[id][7]}</li>
            <li>第9題: {comment_dict[id][8]}</li>
            <li>第10題: {comment_dict[id][9]}</li>
        </ul>
    </body>
    </html>
    """
    
    # 添加 HTML 內容到郵件
    msg.attach(MIMEText(html_content, 'html'))

    # 發送郵件
    try:
        # 連接 SMTP 服務器
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()  # 開啟安全傳輸模式
        server.login(sender_email, sender_password)
        server.send_message(msg)  # 發送郵件
        #print(f"{id} 郵件發送成功！")
    except Exception as e:
        print(f"{id} 發送失敗：{e}")
    finally:
        server.quit()